In [3]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("resume.pdf")
docs = loader.load()
docs

[Document(page_content='Bharat Kumar Sharma\n♂phone+91-8077860792 /envel⌢peiambharatks@gmail.com /linkedinlinkedin.com/in/iambharatks /githubgithub.com/iambharatks\nCodeforces Profile Leetcode Profile Codechef Profile\nEducation\nKIET Group Of Institutions Aug, 2019 – Aug, 2023\nBachelor of Technology in Computer Science and Engineering 89.4%\nExperience\nInfosys October 2023 – Present\nSpecialist Programmer (Java Full Stack and AI) Hyderabad, India\n• Customer Retail and Logistics Team: Engineered robust and scalable REST and GraphQL APIs using Java\nwith the Spring Boot framework. Designed and implemented caching mechanisms using Redis , enhancing\nresponse times for frequently accessed data. Implemented schedulers to precompute data , reducing on-the-fly\ncomputation time and lowering application latency by 30%. Participated in architecture design meetings with\ntechnical architects, contributing to system design and scalability discussions. Contributed to Proof of Concepts for\nfin

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content='Bharat Kumar Sharma\n♂phone+91-8077860792 /envel⌢peiambharatks@gmail.com /linkedinlinkedin.com/in/iambharatks /githubgithub.com/iambharatks\nCodeforces Profile Leetcode Profile Codechef Profile\nEducation\nKIET Group Of Institutions Aug, 2019 – Aug, 2023\nBachelor of Technology in Computer Science and Engineering 89.4%\nExperience\nInfosys October 2023 – Present\nSpecialist Programmer (Java Full Stack and AI) Hyderabad, India\n• Customer Retail and Logistics Team: Engineered robust and scalable REST and GraphQL APIs using Java\nwith the Spring Boot framework. Designed and implemented caching mechanisms using Redis , enhancing\nresponse times for frequently accessed data. Implemented schedulers to precompute data , reducing on-the-fly\ncomputation time and lowering application latency by 30%. Participated in architecture design meetings with\ntechnical architects, contributing to system design and scalability discussions. Contributed to Proof of Concepts for', me

In [5]:
documents=text_splitter.split_documents(docs)
documents

[Document(page_content='Bharat Kumar Sharma\n♂phone+91-8077860792 /envel⌢peiambharatks@gmail.com /linkedinlinkedin.com/in/iambharatks /githubgithub.com/iambharatks\nCodeforces Profile Leetcode Profile Codechef Profile\nEducation\nKIET Group Of Institutions Aug, 2019 – Aug, 2023\nBachelor of Technology in Computer Science and Engineering 89.4%\nExperience\nInfosys October 2023 – Present\nSpecialist Programmer (Java Full Stack and AI) Hyderabad, India\n• Customer Retail and Logistics Team: Engineered robust and scalable REST and GraphQL APIs using Java\nwith the Spring Boot framework. Designed and implemented caching mechanisms using Redis , enhancing\nresponse times for frequently accessed data. Implemented schedulers to precompute data , reducing on-the-fly\ncomputation time and lowering application latency by 30%. Participated in architecture design meetings with\ntechnical architects, contributing to system design and scalability discussions. Contributed to Proof of Concepts for', me

In [6]:
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

db=FAISS.from_documents(documents[:30],OllamaEmbeddings())

In [7]:
db

In [8]:
query="Provide the educational qualification"
result=db.similarity_search(query)
result[0].page_content

'Bharat Kumar Sharma\n♂phone+91-8077860792 /envel⌢peiambharatks@gmail.com /linkedinlinkedin.com/in/iambharatks /githubgithub.com/iambharatks\nCodeforces Profile Leetcode Profile Codechef Profile\nEducation\nKIET Group Of Institutions Aug, 2019 – Aug, 2023\nBachelor of Technology in Computer Science and Engineering 89.4%\nExperience\nInfosys October 2023 – Present\nSpecialist Programmer (Java Full Stack and AI) Hyderabad, India\n• Customer Retail and Logistics Team: Engineered robust and scalable REST and GraphQL APIs using Java\nwith the Spring Boot framework. Designed and implemented caching mechanisms using Redis , enhancing\nresponse times for frequently accessed data. Implemented schedulers to precompute data , reducing on-the-fly\ncomputation time and lowering application latency by 30%. Participated in architecture design meetings with\ntechnical architects, contributing to system design and scalability discussions. Contributed to Proof of Concepts for'

In [9]:
from langchain_community.llms import Ollama
## Load Ollama LAMA2 LLM model
llm=Ollama(model="llama2")
llm

Ollama()

In [10]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}""")

In [11]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain=create_stuff_documents_chain(llm,prompt)

In [12]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever=db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000020C05208D90>)

In [13]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

In [14]:
response=retrieval_chain.invoke({"input":"Provide the educational qualification"})

In [15]:
response

{'input': 'Provide the educational qualification',
 'context': [Document(page_content='Bharat Kumar Sharma\n♂phone+91-8077860792 /envel⌢peiambharatks@gmail.com /linkedinlinkedin.com/in/iambharatks /githubgithub.com/iambharatks\nCodeforces Profile Leetcode Profile Codechef Profile\nEducation\nKIET Group Of Institutions Aug, 2019 – Aug, 2023\nBachelor of Technology in Computer Science and Engineering 89.4%\nExperience\nInfosys October 2023 – Present\nSpecialist Programmer (Java Full Stack and AI) Hyderabad, India\n• Customer Retail and Logistics Team: Engineered robust and scalable REST and GraphQL APIs using Java\nwith the Spring Boot framework. Designed and implemented caching mechanisms using Redis , enhancing\nresponse times for frequently accessed data. Implemented schedulers to precompute data , reducing on-the-fly\ncomputation time and lowering application latency by 30%. Participated in architecture design meetings with\ntechnical architects, contributing to system design and sca

In [16]:
response['answer']

"Based on the provided context, the candidate's educational qualification is:\n\n* Bachelor of Technology in Computer Science and Engineering from KIET Group Of Institutions (August 2019 - August 2023)\n\nThe candidate has completed their undergraduate degree in computer science and engineering and has gained experience in various fields such as software development, NLP, ML, and automation."

In [17]:
response=retrieval_chain.invoke({"input":"Provide the list of skills the applicant is well versed with"})

In [18]:
response['answer']

'Based on the provided context, the following are the skills that the applicant, Bharat Kumar Sharma, is well-versed with:\n\n1. Programming languages: C++, Java, Dart, Python, HTML/CSS, JavaScript, SQL\n2. Developer tools: VS Code, Jupyter Notebook, Ollama, Eclipse, Postman, Android Studio, IntelliJ, Anaconda, Jira, Trello\n3. Technologies/Frameworks: Spring Boot, Mockito, JUnit, Redis, LangChain, LangGraph, LlamaIndex, Pytorch, Hugging Face, Microsoft Azure, Machine Learning, Deep Learning, Natural Language Processing, MongoDB, Selenium\n4. Relevant coursework: Data Structures, Algorithms Analysis, Database Management System, Artificial Intelligence, Object Oriented Programming, Operating Systems, Design Patterns, Generative AI\n5. Achievements: Global rank in various coding competitions such as Google Code Jam Qualification Round 2022, Google Kickstart Round A March 2022, Meta Hackercup 2022, TCS Codevita 2022, GeeksforGeeks Job-A-Thon 5 and 7, Codechef Contests.\n\nNote that this l

In [19]:
print(response['answer'])

Based on the provided context, the following are the skills that the applicant, Bharat Kumar Sharma, is well-versed with:

1. Programming languages: C++, Java, Dart, Python, HTML/CSS, JavaScript, SQL
2. Developer tools: VS Code, Jupyter Notebook, Ollama, Eclipse, Postman, Android Studio, IntelliJ, Anaconda, Jira, Trello
3. Technologies/Frameworks: Spring Boot, Mockito, JUnit, Redis, LangChain, LangGraph, LlamaIndex, Pytorch, Hugging Face, Microsoft Azure, Machine Learning, Deep Learning, Natural Language Processing, MongoDB, Selenium
4. Relevant coursework: Data Structures, Algorithms Analysis, Database Management System, Artificial Intelligence, Object Oriented Programming, Operating Systems, Design Patterns, Generative AI
5. Achievements: Global rank in various coding competitions such as Google Code Jam Qualification Round 2022, Google Kickstart Round A March 2022, Meta Hackercup 2022, TCS Codevita 2022, GeeksforGeeks Job-A-Thon 5 and 7, Codechef Contests.

Note that this list may n